# Structured Output

## There are many way to make structure output.

In [1]:
from pydantic import BaseModel, Field


class ResponseFormatter(BaseModel):
    answer: str = Field(description="The answer to the user's question")
    followup_questions: str = Field(
        description="A followup question the user could ask.")

### Using tool calling

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Bind ResponseFormatter schema as tool to the model
model_with_tools = model.bind_tools([ResponseFormatter])

ai_message = model_with_tools.invoke("what is the powerhouse of cell")

In [7]:
ai_message.tool_calls[0]["args"]

# Parse the dictionary into a pydantic object
pydantic_object = ResponseFormatter.model_validate(
    ai_message.tool_calls[0]["args"])

print(pydantic_object)

answer="The powerhouse of the cell is the mitochondrion. Mitochondria are organelles that generate most of the cell's supply of adenosine triphosphate (ATP), which is used as a source of chemical energy. They are also involved in other important processes such as the regulation of the cell cycle and cell growth." followup_questions='What are the functions of mitochondria?'


### JSON mode

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini").with_structured_output(method="json_mode")

ai_msg = model.invoke(
    "Return a JSON object with key 'random_ints' and a value of 10 random ints in [0-99]")

ai_msg

{'random_ints': [34, 72, 18, 56, 89, 7, 43, 21, 66, 5]}

### Structured output method

In [9]:
from langchain_openai import ChatOpenAI

model_with_structure = ChatOpenAI(
    model="gpt-4o-mini").with_structured_output(ResponseFormatter)

# Invoke the model
structured_output = model_with_structure.invoke(
    "What is the powerhouse of the cell?")

# Get back the pydantic object
structured_output

ResponseFormatter(answer="The powerhouse of the cell is the mitochondrion (plural: mitochondria). Mitochondria are organelles that generate most of the cell's supply of adenosine triphosphate (ATP), used as a source of chemical energy. They are often referred to as the powerhouse because they play a crucial role in energy production through the process of cellular respiration.", followup_questions='What other functions do mitochondria have in the cell?')